# hanlp的分词 接上 BPE 分词

In [1]:
import os
import pickle
import nltk
from data import Vocabulary


c:\Users\liste\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
from data import Preprocessor
# 1. 加载数据,做tokenize和clean
nltk.data.find('tokenizers/punkt')
nltk.data.find('tokenizers/punkt_tab')
print('正在加载和预处理数据...')
preprocessor = Preprocessor()

# 加载训练数据
train_files = [os.path.join('dataset','train_100k.jsonl')]
train_data = preprocessor.load_data(train_files)
# 在这里做tokenize和clean
#train_pairs = preprocessor.prepare_data(train_data)

#print(f'加载完成，训练数据对数量: {len(train_pairs)}')


正在加载和预处理数据...


In [ ]:
def merge_vocab(self, pair, vocab):
    v_out = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in vocab:
        w_out = word.replace(bigram, replacement)
        v_out[w_out] = vocab[word]
    return v_out

In [35]:
# 3. 改进的BPE分词
class SimpleBPE:
    def __init__(self, texts,num_merges=100):
        self.num_merges = num_merges
        self.merges = {}
        self.vocab = Counter()
        self.num_merged = 0
        for text in texts:
            words = text.split()
            for word in words:
                # 在字符间添加空格，除了单词边界
                token = ' '.join(list(word))
                self.vocab[token] += 1

    def get_stats(self, vocab):
        pairs = Counter()
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        v_out = {}
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        for word in vocab:
            w_out = word.replace(bigram, replacement)
            v_out[w_out] = vocab[word]
        return v_out

    def fit(self):
        # BPE合并过程
        vocab=self.vocab
        while self.num_merged < self.num_merges:
            pairs = self.get_stats(vocab)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            self.merges[best_pair] = self.num_merged
            # 输出每次合并的选择
            print(self.num_merged,"th merge: ",best_pair,"freq:",pairs.get(best_pair))
            vocab = self.merge_vocab(best_pair, vocab)
            self.num_merged+=1

        self.vocab = vocab
        return self

    def tokenize(self, text):
        words = text.split()
        tokens = []
        for word in words:
            token = ' '.join(list(word)) 
            for pair in self.merges:
                token = token.replace(' '.join(pair), ''.join(pair))
            tokens.extend(token.split())
        return tokens

In [9]:
import re
def clean_text( text: str) -> str:
    """
    文本清洗：保留有效字符，移除噪声
    注意：移除标点（ BPE不要处理标点 ）
    """

    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', ' ', text)  # 移除控制字符（必须）
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
cleaned_texts=[clean_text(sentence) for sentence in zh_sentences]


In [40]:
zh_sentences =  [pair['zh'] for pair in train_data]
print(zh_sentences[0:2])
print(cleaned_texts[0:2])

['1929年还是1989年?', '巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。']
['1929年还是1989年', '巴黎 随着经济危机不断加深和蔓延 整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况']


In [108]:
for i in cleaned_texts:
    x = i.find('美国家')
    if x>-1:
        print(i)

在次贷危机爆发之后 美国家庭只能用储蓄来支付抵押贷款和消费者债务 或者被迫放弃偿还
巴西和阿根廷这个月承认了 巴勒斯坦 预计将引发一连串拉美国家的纷纷仿效
2008年9月 美国总统小布什希望 解放银行以使信用重新流向美国家庭和企业
最底层四分之一的美国家庭在过去25年中的实际 经通胀调整的 收入几乎没有增长
而额外产生的1万亿美元商品和服务中有一半会表现为美国家庭的消费品和服务
特朗普的这笔交易将把属于中国的这一部分美国多边失衡问题转移到其他地方的更高成本进口商品上 这形同于给美国家庭的加税
一些美国国会议员还想削减食品券 大约2 300万美国家庭需要仰仗食品券 这将令去贫穷儿童面临挨饿的危险
众多美国家庭正在受害 而选举产生的官员则希望把指责推到中国身上 并以此把民众的注意力从重建储蓄 恢复竞争力以及量入为出等艰巨任务上引开
但估计会在11月6号过后继续高悬在美国家庭头上的压力可不会消失
美国和拉美国家都是 征服 社会 是由欧洲人统治的多种族社会
在美国和许多拉美国家 蓄奴也是大规模暴力的助长原因
相反 赚快钱的心态诱使美国家庭将他们的按揭重新融资并支出部分资本从而刺激美国经济
美国家庭正在竭尽全力重塑自己的资产负债表 而资本投资相当健康 因此唯一能够刺激提振生产利用率支出的地方只剩下出口 政府购买和建设投资了
相反 美国应该为美国经济所需要而且想要的人们临时或者永久进入美国建立人道的 安 的以及合法的机制 而且 美国应该与拉美国家政府合作 而不应该与之作对
移民问题总是美国国内的一个极为复杂和微妙的问题 现在对于拉美国家也是一样
尽管如此 从2011年11月国际原子能机构报告详细阐述伊朗追求核武器制造能力的所作所为以来 俄罗斯和中国对于伊朗的态度依然比欧美国家更为宽松
这两大泡沫其实早已破裂 美国家庭正被迫面对泡沫破灭所带来的金融后果 相关问题包括水下资产 创记录的负债和深度储蓄短缺
于是乎 美国家庭前所未有地放低了姿态
此举无异于向困境重重的美国家庭加税 绝不是美国政客所承诺的解决方案
不幸的是 大部分美国家庭的资产负债表的资产端还远远没有复苏
但与其说困扰美国的是暂时的周期性痼疾 还不如说是另一种顽疾的并发症 一场旷日持久的资产负债表衰退持续绑架美国家庭 而家庭消费占到美国国内生产总值约70 的份额
美国家庭曾信手拈来的房地产和信贷两大泡沫早已破灭
其他措施

In [86]:
newbpe=SimpleBPE(cleaned_texts,num_merges=4)
newbpe.fit()
new_tokenized_texts = [newbpe.tokenize(text) for text in cleaned_texts]


0 th merge:  ('经', '济') freq: 12469
1 th merge:  ('中', '国') freq: 11235
2 th merge:  ('美', '国') freq: 11135
3 th merge:  ('国', '家') freq: 10297


In [74]:
new_tokenized_texts[1000]

['但',
 '后',
 '来',
 '他',
 '们',
 '逐',
 '渐',
 '意',
 '识',
 '到',
 '所',
 '探',
 '测',
 '到',
 '的',
 '信',
 '号',
 '可能',
 '完',
 '全',
 '来',
 '源',
 '于',
 '星',
 '际',
 '尘',
 '埃']

In [87]:

newvocab = build_vocabulary(new_tokenized_texts)

In [92]:
for i in list(newvocab):
    if len(i)>1:
        print(i)

经济
国家
美国
中国
中国家
美国家


In [41]:
from collections import Counter
bpe_tokenizer = SimpleBPE(cleaned_texts,num_merges=6000)
bpe_tokenizer.fit()

0 th merge:  ('经', '济') freq: 12469
1 th merge:  ('中', '国') freq: 11235
2 th merge:  ('美', '国') freq: 11135
3 th merge:  ('国', '家') freq: 10297
4 th merge:  ('2', '0') freq: 8133
5 th merge:  ('一', '个') freq: 7765
6 th merge:  ('欧', '洲') freq: 7628
7 th merge:  ('我', '们') freq: 7272
8 th merge:  ('可', '能') freq: 6905
9 th merge:  ('政', '府') freq: 6791
10 th merge:  ('他', '们') freq: 6163
11 th merge:  ('问', '题') freq: 6106
12 th merge:  ('这', '一') freq: 6027
13 th merge:  ('世', '界') freq: 5692
14 th merge:  ('可', '以') freq: 5503
15 th merge:  ('全', '球') freq: 5337
16 th merge:  ('增', '长') freq: 5288
17 th merge:  ('政', '策') freq: 5268
18 th merge:  ('这', '些') freq: 5125
19 th merge:  ('政', '治') freq: 4829
20 th merge:  ('已', '经') freq: 4703
21 th merge:  ('没', '有') freq: 4597
22 th merge:  ('欧', '盟') freq: 4486
23 th merge:  ('需', '要') freq: 4433
24 th merge:  ('市', '场') freq: 4322
25 th merge:  ('银', '行') freq: 4141
26 th merge:  ('其', '他') freq: 4043
27 th merge:  ('因', '为') freq: 398

In [109]:
import pickle
with open('bpe_tokenizer6000.pkl','wb') as f:
    pickle.dump(bpe_tokenizer,f)


In [43]:
bpe_tokenized_texts = [bpe_tokenizer.tokenize(text) for text in cleaned_texts]
print(bpe_tokenized_texts[2])

['一开始', '很多人', '把', '这次', '危机', '比', '作', '198', '2年', '或', '197', '3年', '所', '发生的情况', '这样', '得', '类', '比', '是', '令人', '宽', '心', '的', '因为这两', '段', '时期', '意味着', '典型', '的', '周期性', '衰退']


In [44]:
# 4. 构建词表
def build_vocabulary(tokenized_texts):
    """
    构建词表，并统计每个词的频率
    """
    all_words = []
    for tokens in tokenized_texts:
        all_words.extend(tokens)
    return Counter(all_words)

In [45]:
bpe_vocab = build_vocabulary(bpe_tokenized_texts)

In [46]:
len(bpe_vocab)

58187

In [47]:
bpe_tokenizer.vocab

{'19 29年 还是 1989年': 1,
 '巴黎': 78,
 '随着 经济危机 不断 加深 和 蔓延': 1,
 '整个 世界 一直在 寻找 历史上的 类似 事件 希望 有助于 我们 了解 目前 正在 发生的情况': 1,
 '一开始': 31,
 '很多人 把 这次 危机 比 作 198 2年 或 197 3年 所 发生的情况': 1,
 '这样 得 类 比 是 令人 宽 心 的': 1,
 '因为这两 段 时期 意味着 典型 的 周期性 衰退': 1,
 '如今 人们的 心 情 却是 沉重 多 了': 1,
 '许多人 开始 把 这次 危机 与 19 29年 和 193 1年 相比': 1,
 '即使 一些国家 政府的 表现 仍然 似乎 把 视 目前的情况 为 是 典型 的 而 看 见的 衰退': 1,
 '目前的趋势 是': 1,
 '要么 是 过度的 克 制': 1,
 '欧洲': 44,
 '要么 是 努力的 扩展': 1,
 '美国': 91,
 '欧洲 在 避免 债务 和 捍卫 欧元的名义 下 正 变得 谨慎': 1,
 '而美国 已经在许多 方面 行动起来': 1,
 '以 利用 这一 理想 的 时机 来 实行 急需的 结构性改革': 1,
 '然而': 221,
 '作为 地 域 战略 学家': 1,
 '无论是 从 政治 意义 还是 从 经济 意义上': 1,
 '让 我 自然 想 到的年 份 是 1989年': 1,
 '当然': 711,
 '雷曼 兄弟 公司的 倒闭 和 柏林 墙 的 倒 塌 没有任何 关系': 1,
 '事实上': 1647,
 '从 表面上 看': 4,
 '两者 似乎是 完全是 相反 的': 1,
 '一个 是 象征 着 压 抑 和 人为 分裂 的 柏林 墙 的 倒 塌': 1,
 '而 另一个 是 看似 坚 不可 摧 的 并 令人 安 心 的金融资本主义 机构的 倒 塌': 1,
 '和 1989年 一样': 1,
 '2008': 30,
 '2009年 很可能 也能 被视为 一个 划 时代的 改变': 1,
 '其 带来的 发 人 深 省 的后果 将在 几十年 后 仍 能让我们 感 受 得到': 1,
 '东西方 意识形态 鸿 沟 的 结束': 1,
 '以及对 市场 绝对 信心 的后果': 1,
 '都

In [60]:
len(list(bpe_vocab))

58187

In [57]:
list(bpe_tokenizer.merges)[1900:2000]

[('私人', '部门'),
 ('反映', '了'),
 ('海', '洋'),
 ('准', '确'),
 ('改革', '的'),
 ('瑞', '典'),
 ('借', '款'),
 ('绝', '对'),
 ('一', '起'),
 ('被', '认为'),
 ('将', '继续'),
 ('极', '为'),
 ('居', '民'),
 ('条', '款'),
 ('荷', '兰'),
 ('脱', '离'),
 ('著', '名'),
 ('斯', '特'),
 ('东', '亚'),
 ('瑞', '拉'),
 ('各国', '政府'),
 ('尔', '德'),
 ('不', '太'),
 ('比', '率'),
 ('不', '愿'),
 ('海', '外'),
 ('短', '缺'),
 ('生活', '在'),
 ('l', 'e'),
 ('历史', '上'),
 ('边', '界'),
 ('协', '助'),
 ('委', '内'),
 ('委内', '瑞拉'),
 ('加', '拿'),
 ('让', '人'),
 ('都', '能'),
 ('以', '前'),
 ('但', '他们'),
 ('特', '殊'),
 ('决', '心'),
 ('新', '加坡'),
 ('以', '此'),
 ('电', '力'),
 ('所', '在'),
 ('绝', '非'),
 ('过', '剩'),
 ('的', '数据'),
 ('部', '队'),
 ('超', '级'),
 ('安', '倍'),
 ('牺', '牲'),
 ('客', '户'),
 ('最', '新'),
 ('意', '见'),
 ('付', '出'),
 ('示', '威'),
 ('要', '比'),
 ('减', '速'),
 ('尚', '未'),
 ('越', '南'),
 ('还', '需要'),
 ('的', '市场'),
 ('温室', '气体'),
 ('以', '后'),
 ('流', '入'),
 ('法', '案'),
 ('中', '间'),
 ('出', '于'),
 ('渠', '道'),
 ('组', '合'),
 ('看', '起来'),
 ('极', '大'),
 ('入', '侵'),
 ('低', '收入'),
 ('工